In [1]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot = True)

#定义每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
#初始化权值
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial,name=name)

#初始化偏置值
def bias_variable(shape,name):
    initial = tf.zeros(shape)+0.1
    return tf.Variable(initial,name=name)

#卷积层
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

#池化层
def maxpool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('input'):
    #定义两个placeholder
    x=tf.placeholder(tf.float32,[None,784],name="x-input")
    y=tf.placeholder(tf.float32,[None,10],name="y-input")
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    with tf.name_scope('x_image'):
        x_image = tf.reshape(x,[-1,28,28,1],name='x_image')

with tf.name_scope('conv_1'):
    with tf.name_scope('w_conv1'):
        #对第一卷积层的filter和bias做初始化
        w_conv1 = weight_variable([5,5,1,32],name='w_conv1')
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32],name='b_conv1')

    with tf.name_scope('conv2d_1'):
        #把x_image和filter做卷积，在加上bias，作用于relu激活函数
        conv2d_1 = conv2d(x_image,w_conv1)+b_conv1
    with tf.name_scope('relu_1'):
        relu_1 = tf.nn.relu(conv2d_1)
    with tf.name_scope('pool_1'):
        pool_1 = maxpool(relu_1)

with tf.name_scope('conv_2'):
    with tf.name_scope('w_conv2'):
        #对第二卷积层的filter和bias做初始化
        w_conv2 = weight_variable([5,5,32,64],name='w_conv2')
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64],name='b_conv2')

    with tf.name_scope('conv2d_2'):
        #吧第一层输出和filter做卷积，在加上bias，作用于relu函数
        conv2d_2 = conv2d(pool_1,w_conv2)+b_conv2
    with tf.name_scope('relu_2'):
        relu_2 = tf.nn.relu(conv2d_2)
    with tf.name_scope('pool_2'):
        pool_2 = maxpool(relu_2)

with tf.name_scope('fc_1'):
    with tf.name_scope('pool_2_flat'):
        #把池化层2的输出扁平化为1维
        pool_2_flat = tf.reshape(pool_2,[-1,7*7*64],name='pool_2_flat')

    with tf.name_scope('w_fc1'):
        #初始化第一个全连接层的权值和bias 
        w_fc1 = weight_variable([7*7*64,1024],name='w_fc1')
    with tf.name_scope('b_fc1'):
        b_fc1 = bias_variable([1024],name='b_fc1')

    with tf.name_scope('matmul_1'):
        #求第一个全连接层的输出
        matmul_1 = tf.matmul(pool_2_flat,w_fc1)+b_fc1
    with tf.name_scope('relu_3'):
        relu_3 = tf.nn.relu(matmul_1)
    with tf.name_scope('fc_1_drop'):
        fc_l_drop = tf.nn.dropout(relu_3,keep_prob)

with tf.name_scope('fc_2'):
    with tf.name_scope('w_fc2'):
        #初始化第二个全连接层的权值和bias
        w_fc2 = weight_variable([1024,10],name='w_fc2')
    with tf.name_scope('b_fc2'):
        b_fc2 = bias_variable([10],name='b_fc2')
        
    with tf.name_scope('matmul_2'):
        matmul_2 = tf.matmul(relu_3,w_fc2)+b_fc2
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(matmul_2)

with tf.name_scope('loss'):
    #交叉熵代价函数 loss指标表示模型的好壞
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train_step'):
    #使用梯度下降法
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
    with tf.name_scope('accuracy'):
        #计算准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
#合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter('logs/train',sess.graph)
    test_writer = tf.summary.FileWriter('logs/test',sess.graph)
    for epoch in range(1001):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.5})
        summary = sess.run(merged,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        train_writer.add_summary(summary,epoch)
        summary = sess.run(merged,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        test_writer.add_summary(summary,epoch)
        
        if epoch%100==0:
            training_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
            testing_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            print("iter"+str(epoch)+",testing accuracy"+str(testing_acc)+",training accuracy"+str(training_acc))